In [9]:
from grpc.beta import implementations
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import scipy.misc
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [80]:
PATH_TO_LABELS = '/notebooks/data/pet_label_map.pbtxt'
SERVING_HOST = 'object-detection.21-kuberlab-demo.svc.cluster.local'
IMAGE_FILE = '/notebooks/pet.jpg'
OUTPUT_FILE = '/notebooks/outfile.png'
MODEL_NAME = 'object-detection-pets'
IMAGE_SIZE = (18, 12)

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [12]:
data = open(IMAGE_FILE, 'rb').read() # Binary data to serving
image = Image.open(IMAGE_FILE) # Image will be used for drawing detection boxes
image_np = load_image_into_numpy_array(image)
image_np_expanded = np.expand_dims(image_np, axis=0)
proto = tf.contrib.util.make_tensor_proto(data, shape=[1])

request = predict_pb2.PredictRequest()
request.model_spec.name = MODEL_NAME
request.inputs['inputs'].ParseFromString(proto.SerializeToString())
channel = implementations.insecure_channel(SERVING_HOST, 9000)
stub = prediction_service_pb2.beta_create_PredictionService_stub(channel)

In [20]:
resp = stub.Predict(request, 10.0)

In [26]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
max_num_classes = max([item.id for item in label_map.item])

categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes)
category_index = label_map_util.create_category_index(categories)

In [76]:
# Convert tensorflow output protos to dict
output_dict = {}
for key in resp.outputs:
    tensor_proto = resp.outputs[key]
    nd_array = tf.contrib.util.make_ndarray(tensor_proto)
    output_dict[key] = nd_array

# Convert each value to appropriate type
output_dict['num_detections'] = int(output_dict['num_detections'][0])
output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.uint8)
output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
output_dict['detection_scores'] = output_dict['detection_scores'][0]
if 'detection_masks' in output_dict:
    output_dict['detection_masks'] = output_dict['detection_masks'][0]

In [77]:
# Visualize
%matplotlib inline
image_with_boxes = vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8
)
plt.figure(figsize=IMAGE_SIZE)
plt.imshow(image_np)

In [ ]:
# Determine name if it is not seen quite good.
print(category_index[output_dict['detection_classes'][0]]['name'])

In [ ]:
# Save image so we can download it
scipy.misc.imsave(OUTPUT_FILE, image_np)